In [3]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import os
import sys
import platform
from copy import deepcopy
from typing import List, Dict, Optional, Union, Callable
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
import tables as tb
from copy import deepcopy
from datetime import datetime, timedelta
from attrs import define, field, Factory

# # required to enable non-blocking interaction:
# %gui qt5 ## TODO 2024-01-18 - this causes kernel to crash when running notebook remotely via VSCode's ssh remote

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.exception_helpers import CapturedException

# Jupyter interactivity:
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.JupyterButtonRowWidget import JupyterButtonRowWidget
from pyphocorehelpers.gui.Jupyter.simple_widgets import code_block_widget

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core import Epoch

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, BatchComputationProcessOptions, BatchSessionCompletionHandler, SavingOptions
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.General.Batch.runBatch import SessionBatchProgress
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionTables, AcrossSessionsVisualizations

from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import build_vscode_workspace, build_windows_powershell_run_script

from pyphocorehelpers.exception_helpers import CapturedException
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult, BatchSessionCompletionHandler

from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots, BatchPhoJonathanFiguresHelper
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

from neuropy.core.neuron_identities import NeuronIdentityDataframeAccessor
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import build_merged_neuron_firing_rate_indicies
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsHelpers
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, filesystem_path_folder_contents_widget

import inspect
from jinja2 import Template
from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import MAIN_get_template_string, write_test_script
from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import export_session_h5_file_completion_function, curr_runtime_context_header_template, export_rank_order_results_completion_function, figures_rank_order_results_completion_function, compute_and_export_marginals_dfs_completion_function, determine_session_t_delta_completion_function, perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function, compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function, reload_exported_kdiba_session_position_info_mat_completion_function, compute_and_export_session_wcorr_shuffles_completion_function, compute_and_export_session_instantaneous_spike_rates_completion_function, compute_and_export_session_extended_placefield_peak_information_completion_function
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import ProcessingScriptPhases

# BATCH_DATE_TO_USE = '2024-06-12_GL' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = '2024-06-12_Lab' # used for filenames throught the notebook
BATCH_DATE_TO_USE = '2024-06-12_Apogee' # used for filenames throught the notebook

# scripts_output_path = Path('/home/halechr/cloud/turbo/Data/Output/gen_scripts/').resolve() # Greatlakes
# scripts_output_path = Path('output/gen_scripts/').resolve() # Apogee
# # scripts_output_path = Path('/home/halechr/FastData/gen_scripts/').resolve() # Lab
# assert scripts_output_path.exists()
known_scripts_output_paths = [Path(v).resolve() for v in ['/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs','/home/halechr/cloud/turbo/Data/Output/gen_scripts/', '/home/halechr/FastData/gen_scripts/', 'output/gen_scripts/']]
scripts_output_path = find_first_extant_path(known_scripts_output_paths)
assert scripts_output_path.exists(), f"scripts_output_path: {scripts_output_path} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'scripts_output_path: {scripts_output_path}')

collected_outputs_path = scripts_output_path.joinpath('../collected_outputs').resolve()
collected_outputs_path.mkdir(exist_ok=True)
assert collected_outputs_path.exists()
print(f'collected_outputs_path: {collected_outputs_path}')

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
scripts_output_path: /Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs
collected_outputs_path: /Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs


## Build Processing Scripts:

In [4]:
# Get the generated code for user-contributed functions:
phase_any_run_custom_user_completion_functions_dict = {
# "export_rank_order_results_completion_function": export_rank_order_results_completion_function, # ran 2024-05-28 6am
# "figures_rank_order_results_completion_function": figures_rank_order_results_completion_function,
# "compute_and_export_marginals_dfs_completion_function": compute_and_export_marginals_dfs_completion_function, # ran 2024-05-28 6am
# "determine_session_t_delta_completion_function": determine_session_t_delta_completion_function,  # ran 2024-05-28 6am
# 'perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function': perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function, # ran 2024-05-28 6am
# 'compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function': compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function, # ran 2024-05-28 6am
# 'reload_exported_kdiba_session_position_info_mat_completion_function': reload_exported_kdiba_session_position_info_mat_completion_function,
'export_session_h5_file_completion_function': export_session_h5_file_completion_function, # ran 2024-05-28 3am
    # 'compute_and_export_session_wcorr_shuffles_completion_function': compute_and_export_session_wcorr_shuffles_completion_function,
'compute_and_export_session_instantaneous_spike_rates_completion_function': compute_and_export_session_instantaneous_spike_rates_completion_function,
'compute_and_export_session_extended_placefield_peak_information_completion_function': compute_and_export_session_extended_placefield_peak_information_completion_function,
}


In [5]:
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import BatchScriptsCollection, generate_batch_single_session_scripts, display_generated_scripts_ipywidget

# Hardcoded included_session_contexts:
included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # Other file
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), 
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # 
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # Other tab
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'), 
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), #
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), #
]

## Setup Functions:
force_recompute_override_computations_includelist = []
# force_recompute_override_computations_includelist = ['directional_decoders_evaluate_epochs',]
# force_recompute_override_computations_includelist = ['split_to_directional_laps','merged_directional_placefields','directional_decoders_evaluate_epochs','directional_decoders_epoch_heuristic_scoring']
# force_recompute_override_computations_includelist = ['directional_decoders_evaluate_epochs','directional_decoders_epoch_heuristic_scoring']
# force_recompute_override_computations_includelist = ['split_to_directional_laps','merged_directional_placefields']
# force_recompute_override_computations_includelist = ['long_short_decoding_analyses', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding',]
# force_recompute_override_computations_includelist = ['wcorr_shuffle_analysis','trial_by_trial_metrics']

extra_extended_computations_include_includelist = [
	# 'wcorr_shuffle_analysis'
]

# custom_phase_extended_computations_include_includelist = None
custom_phase_extended_computations_include_includelist=['lap_direction_determination', 'pf_computation', 'pfdt_computation', 'firing_rate_trends',
	# 'pf_dt_sequential_surprise',
	'extended_stats',
	'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 
	'ratemap_peaks_prominence2d',
	'long_short_inst_spike_rate_groups',
	'long_short_endcap_analysis',
	# 'spike_burst_detection',
	'split_to_directional_laps',
	'merged_directional_placefields',
	'rank_order_shuffle_analysis',
	# 'directional_train_test_split',
	'directional_decoders_decode_continuous',
	'directional_decoders_evaluate_epochs',
	'directional_decoders_epoch_heuristic_scoring',
    # 'wcorr_shuffle_analysis',
    'trial_by_trial_metrics',
    'extended_pf_peak_information',
]

active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'

debug_print = False
known_global_data_root_parent_paths = [Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'W:/Data'), Path(r'/home/halechr/FastData'), Path(r'/Volumes/MoverNew/data')] # Path(r'/home/halechr/cloud/turbo/Data'), , Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/home/halechr/turbo/Data'), 
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

## Different run configurations:

# active_phase = ProcessingScriptPhases.clean_run
# active_phase = ProcessingScriptPhases.continued_run
active_phase = ProcessingScriptPhases.final_run
# active_phase = ProcessingScriptPhases.figure_run

custom_user_completion_functions_dict = active_phase.get_custom_user_completion_functions_dict(extra_run_functions=phase_any_run_custom_user_completion_functions_dict)
# print(f'custom_user_completion_functions_dict: {custom_user_completion_functions_dict}')
custom_user_completion_function_template_code, custom_user_completion_functions_dict = MAIN_get_template_string(BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, collected_outputs_path=collected_outputs_path, override_custom_user_completion_functions_dict=custom_user_completion_functions_dict)
# print(f'custom_user_completion_function_template_code: {custom_user_completion_function_template_code}')
active_phase_dict = active_phase.get_run_configuration(custom_user_completion_function_template_code=custom_user_completion_function_template_code, extra_extended_computations_include_includelist=extra_extended_computations_include_includelist)

## Completely replace with custom functions:
if custom_phase_extended_computations_include_includelist is not None:
	print(f'WARNING: custom_phase_extended_computations_include_includelist: {custom_phase_extended_computations_include_includelist} is provided so only these extended_computation_fns will be used (overwritting the phase defaults!).')
	active_phase_dict['extended_computations_include_includelist'] = custom_phase_extended_computations_include_includelist

# active_phase_dict['extended_computations_include_includelist'].remove('wcorr_shuffle_analysis')
# active_phase_dict['should_freeze_pipeline_updates'] = True

## Build Slurm Scripts:
session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}
batch_scripts_collection: BatchScriptsCollection = generate_batch_single_session_scripts(global_data_root_parent_path, session_batch_basedirs=session_basedirs_dict, included_session_contexts=included_session_contexts,
                                                                                        output_directory=scripts_output_path, use_separate_run_directories=True,
                                                                                        # should_freeze_pipeline_updates=False, 
                                                                                        # create_slurm_scripts=True, should_create_vscode_workspace=True,
                                                                                        # extended_computations_include_includelist=extended_computations_include_includelist,
                                                                                        force_recompute_override_computations_includelist=force_recompute_override_computations_includelist, # ['split_to_directional_laps', 'rank_order_shuffle_analysis'],
                                                                                        # should_perform_figure_generation_to_file=False,
                                                                                        # batch_session_completion_handler_kwargs=dict(enable_hdf5_output=False),
                                                                                        # custom_user_completion_functions=custom_user_completion_functions,
                                                                                        # custom_user_completion_function_template_code=custom_user_completion_function_template_code,
                                                                                        # should_force_reload_all=True,
                                                                                        # should_force_reload_all=False,
                                                                                        **active_phase_dict
                                                                                    )


# batch_scripts_collection.included_session_contexts, output_python_scripts, output_slurm_scripts

output_python_scripts = batch_scripts_collection.output_python_scripts
output_slurm_scripts = batch_scripts_collection.output_slurm_scripts
vscode_workspace_path = batch_scripts_collection.vscode_workspace_path
if vscode_workspace_path is not None:
    display(fullwidth_path_widget(vscode_workspace_path, file_name_label='vscode_workspace_path:'))

computation_script_paths = [x[0] for x in output_python_scripts]
generate_figures_script_paths = [x[1] for x in output_python_scripts]
print(f'generate_figures_script_paths: {generate_figures_script_paths}')
_out_widget = display_generated_scripts_ipywidget(batch_scripts_collection.included_session_contexts, output_python_scripts)
display(_out_widget)

FileNotFoundError: Could not find any of the paths in the list: [PosixPath('/nfs/turbo/umms-kdiba/Data'), PosixPath('/media/halechr/MAX/Data'), PosixPath('W:/Data'), PosixPath('/home/halechr/FastData'), PosixPath('/Volumes/MoverNew/data')]

In [4]:
print(active_phase_dict['extended_computations_include_includelist'])

['lap_direction_determination', 'pf_computation', 'pfdt_computation', 'firing_rate_trends', 'extended_stats', 'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'ratemap_peaks_prominence2d', 'long_short_inst_spike_rate_groups', 'long_short_endcap_analysis', 'split_to_directional_laps', 'merged_directional_placefields', 'rank_order_shuffle_analysis', 'directional_decoders_decode_continuous', 'directional_decoders_evaluate_epochs', 'directional_decoders_epoch_heuristic_scoring', 'trial_by_trial_metrics', 'extended_pf_peak_information']


In [5]:
# Maximum number of parallel script executions
max_parallel_executions = 3
# List of your script paths
# if active_phase.value == ProcessingScriptPhases.figure_run:
if active_phase.is_figure_phase:
    print(f'fig mode!')
    script_paths = generate_figures_script_paths
else:
	script_paths = computation_script_paths

if (platform.system() == 'Windows'):
    powershell_script_path = build_windows_powershell_run_script(script_paths, max_concurrent_jobs=max_parallel_executions, script_name='run_scripts')
    # print(f'powershell_script_path: {powershell_script_path}')
    display(fullwidth_path_widget(powershell_script_path, file_name_label='powershell_script_path:'))


    if active_phase_dict['should_perform_figure_generation_to_file']:
        powershell_figures_script_path = build_windows_powershell_run_script(generate_figures_script_paths, max_concurrent_jobs=1, script_name='run_figure_scripts')
        display(fullwidth_path_widget(powershell_figures_script_path, file_name_label='powershell_figures_script_path:'))


if (platform.system() != 'Windows'):
    ## Linux Only: Slurm Scripts
    # sbatch_start_slurm_scripts: str = "\n".join([f"sbatch '{slurm_script}'" for slurm_script in output_slurm_scripts])
    sbatch_run_start_slurm_scripts: str = "\n".join([f"sbatch '{slurm_script}'" for slurm_script in output_slurm_scripts['run']])
    sbatch_figs_start_slurm_scripts: str = "\n".join([f"sbatch '{slurm_script}'" for slurm_script in output_slurm_scripts['figs']])

    if active_phase.is_figure_phase:
        # print(sbatch_figs_start_slurm_scripts)
        initial_code = sbatch_figs_start_slurm_scripts
    else:
        # print(sbatch_run_start_slurm_scripts)
        initial_code = sbatch_run_start_slurm_scripts

    # Create and display the code block widget
    slurm_code_block = code_block_widget(initial_code, label="Python Code:")



# Batch 1
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-02_17-46-44/run_kdiba_pin01_one_11-02_17-46-44.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-02_19-28-0/run_kdiba_pin01_one_11-02_19-28-0.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-03_12-3-25/run_kdiba_pin01_one_11-03_12-3-25.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_fet11-01_12-58-54/run_kdiba_pin01_one_fet11-01_12-58-54.sh'

# Batch 2:
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-09_17-29-30/run_kdiba_vvp01_one_2006-4-09_17-29-30.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-10_12-25-50/run_kdiba_vvp01_one_2006-4-10_12-25-50.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-09_16-40-54/run_kdiba_vvp01_two_2006-4-09_16-40-54.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-10_12-58-3/run_kdiba_vvp01_two_2006-4-10_12-58-3.sh'

# Batch 3
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15/run_kdiba_gor01_one_2006-6-08_14-26-15.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43/run_kdiba_gor01_one_2006-6-09_1-22-43.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19/run_kdiba_gor01_two_2006-6-07_16-40-19.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-08_21-16-25/run_kdiba_gor01_two_2006-6-08_21-16-25.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-09_22-24-40/run_kdiba_gor01_two_2006-6-09_22-24-40.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-12_16-53-46/run_kdiba_gor01_two_2006-6-12_16-53-46.sh'


ps_script_path: K:\scratch\gen_scripts\run_scripts.ps1


Box(children=(Label(value='powershell_script_path:', layout=Layout(width='auto')), HTML(value="<b style='font-…

## Execute the generated scripts in parallel:

In [ ]:
import subprocess
from concurrent.futures import ProcessPoolExecutor, as_completed
import ipywidgets as widgets
from IPython.display import display

## recieves: max_parallel_executions, script_paths
"""
# Maximum number of parallel script executions
max_parallel_executions = 5
# List of your script paths
script_paths = output_python_scripts
"""

# Function to execute a script
def run_script(script_path):
    python_executable = 'python'
    # python_executable = '/home/halechr/Library/VSCode/green/.venv_green/bin/python'
    
    try:
        result = subprocess.run([python_executable, script_path], capture_output=True, text=True)
        return script_path, result.stdout, result.stderr
    except Exception as e:
        return script_path, None, str(e)

# Create a progress bar
progress_bar = widgets.IntProgress(value=0, min=0, max=len(script_paths), description='Running:', bar_style='info')
display(progress_bar)

# Run scripts in parallel with a limit on the number of parallel instances
with ProcessPoolExecutor(max_workers=max_parallel_executions) as executor:
    futures = {executor.submit(run_script, path): path for path in script_paths}
    for future in as_completed(futures):
        script, stdout, stderr = future.result()
        progress_bar.value += 1  # Update the progress bar
        if stderr:
            print(f"Error in {script}: {stderr}")
        else:
            print(f"Completed {script}")

# Progress bar will automatically update as scripts complete



## Resume normal stuff:

In [8]:
active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'

debug_print = False
known_global_data_root_parent_paths = [Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data')
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default
# Hardcoded included_session_contexts:
included_session_contexts = UserAnnotationsManager.get_hardcoded_good_sessions()
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

# Output Paths:
included_h5_paths = [get_file_str_if_file_exists(v.pipeline_results_h5) for v in good_session_concrete_folders]
# copy_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, backup_mode=BackupMethods.RenameInSourceDirectory, only_include_file_types=['local_pkl', 'global_pkl'])
check_output_h5_files(included_h5_paths)
# from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import generate_batch_single_session_scripts

# ## Build Slurm Scripts:
# session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}
# included_session_contexts, output_python_scripts, output_slurm_scripts = generate_batch_single_session_scripts(global_data_root_parent_path, session_batch_basedirs=session_basedirs_dict, included_session_contexts=included_session_contexts, output_directory=Path('output/generated_slurm_scripts/').resolve(), use_separate_run_directories=True, should_perform_figure_generation_to_file=False)
# display(output_python_scripts)

## OUTPUTS: included_h5_paths, included_session_contexts, good_session_concrete_folders

WARN: file_size < 0.01 for W:\Data\KDIBA\gor01\two\2006-6-12_16-53-46\output\pipeline_results.h5!
WARN: file_size < 0.01 for W:\Data\KDIBA\pin01\one\11-02_17-46-44\output\pipeline_results.h5!


,path,modification_time,creation_time,file_size
0,W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\out...,2024-06-12 09:58:19.618948,2023-08-31 22:00:50.256063,1.06
1,W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\outp...,2024-06-12 09:57:48.500054,2023-08-07 22:42:27.913653,0.93
2,W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31\out...,2024-06-12 09:59:32.133657,2023-08-03 18:14:52.394542,0.30
3,W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\out...,2024-06-12 10:04:39.825718,2023-08-03 18:15:07.920659,0.81
4,W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\out...,2024-06-12 10:08:14.334885,2023-08-08 03:20:28.339371,0.50
5,W:\Data\KDIBA\gor01\two\2006-6-09_22-24-40\out...,2024-05-28 02:47:53.174566,2023-08-08 03:20:48.904983,1.25
6,W:\Data\KDIBA\vvp01\one\2006-4-09_17-29-30\out...,2024-06-12 10:03:53.694698,2023-08-08 03:21:51.139954,0.25
7,W:\Data\KDIBA\vvp01\one\2006-4-10_12-25-50\out...,2024-06-12 10:07:39.103949,2023-08-08 03:22:12.043931,0.21
8,W:\Data\KDIBA\vvp01\two\2006-4-09_16-40-54\out...,2024-06-12 10:08:11.561890,2023-08-08 03:22:32.722443,0.25
9,W:\Data\KDIBA\vvp01\two\2006-4-10_12-58-3\outp...,2024-06-12 10:11:44.030021,2023-08-08 03:22:53.658802,0.20


In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import save_filelist_to_text_file

## INPUTS: good_session_concrete_folders, target_dir, BATCH_DATE_TO_USE

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path

included_h5_paths = [Path(get_file_str_if_file_exists(v.pipeline_results_h5)).resolve() for v in good_session_concrete_folders]
check_output_h5_files(included_h5_paths)
included_h5_paths

def _across_session_h5_output_basename_fn(session_context: Optional[IdentifyingContext], session_descr: Optional[str], basename: str, *args, separator_char: str = "_"):
    """ Captures `BATCH_DATE_TO_USE` """
    # a_session_folder.context
    if session_context is not None:
        session_descr = session_context.session_name # '2006-6-07_16-40-19'
    _filename_list = [BATCH_DATE_TO_USE, session_descr, basename]
    if len(args) > 0:
        _filename_list.extend([str(a_part) for a_part in args if a_part is not None])
    return separator_char.join(_filename_list)

copy_h5_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, target_dir=collected_outputs_path, backup_mode=BackupMethods.CommonTargetDirectory, rename_backup_basename_fn=_across_session_h5_output_basename_fn, only_include_file_types=['h5']) # , rename_backup_suffix=BATCH_DATE_TO_USE
copy_h5_dict


## INPUTS: target_dir, BATCH_DATE_TO_USE
h5_filelist_output_filename=f'{BATCH_DATE_TO_USE}_all_sessions_h5_filelist.txt'
h5_filelist_output_file_path = Path(target_dir).joinpath(h5_filelist_output_filename).resolve() # Use Default
print(f'h5_filelist_output_file_path: {h5_filelist_output_file_path}')
_out_string, filelist_path = save_filelist_to_text_file(included_h5_paths, filelist_path=h5_filelist_output_file_path, debug_print=True) # r"W:\Data\all_sessions_h5_filelist_2024-03-28_Apogee.txt"
filelist_path

In [ ]:
copy_h5_dict

In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import read_filelist_from_text_file
    
an_h5_filelist_path = Path(r'L:/Scratch/collected_outputs/2024-03-28_Apogee_all_sessions_h5_filelist.txt').resolve()
read_hdf5_output_paths = read_filelist_from_text_file(filelist_path=an_h5_filelist_path, debug_print=False)
read_hdf5_output_paths


# Output File Processing Helpers

## Copy exported `pipeline_results.h5` files into the `collected_outputs` folder, adding the current date
```
copying "W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\output\pipeline_results.h5"
		 -> "K:\scratch\collected_outputs\2024-06-12_Apogee_2006-6-08_14-26-15_pipeline_results.h5"...
```

In [ ]:
## INPUT a_copy_dict
a_copy_dict = copy_h5_dict

moved_files_dict_h5_files = copy_movedict(a_copy_dict)
moved_files_dict_h5_files
# INPUTS: active_filelist_prefix, target_dir
# active_filelist_prefix: str = 'backed_up_files'
active_filelist_prefix: str = 'session_h5_files'

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path

moved_files_copydict_output_filename=f'{active_filelist_prefix}_copydict_{BATCH_DATE_TO_USE}.csv'
moved_files_copydict_file_path = target_dir.joinpath(moved_files_copydict_output_filename).resolve() # Use Default
print(f'moved_files_copydict_file_path: "{moved_files_copydict_file_path}"')

_out_string, filedict_out_path = save_copydict_to_text_file(moved_files_dict_h5_files, moved_files_copydict_file_path, debug_print=True)

In [ ]:
## INPUTS: moved_files_copydict_file_path
moved_files_copydict_file_path = Path(r"W:\Data\session_h5_files_copydict_2024-03-28_Apogee.csv").resolve()
assert moved_files_copydict_file_path.exists()

read_moved_files_dict_files = read_copydict_from_text_file(moved_files_copydict_file_path, debug_print=False)
read_moved_files_dict_files

In [ ]:
# read_moved_files_dict_files
restore_moved_files_dict_files = invert_filedict(read_moved_files_dict_files)
restore_moved_files_dict_files

# After Sessions are processed and their `.h5` files are exported, build the combined tables
The combined (across session) tables are saved out into .csv and .pkl formats: `W:\Data\2024-06-12_Apogee\neuron_identities_table.csv` -- ideally this would be in the combined outputs folder

In [9]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables

## INPUTS: included_session_contexts, included_h5_paths, global_data_root_parent_path, BATCH_DATE_TO_USE
# neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True, )

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path
assert target_dir.exists(), f'target_dir: "{target_dir}" does not exist!'
(neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table), output_path_dicts = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, included_h5_paths, override_output_parent_path=target_dir, output_path_suffix=f'{BATCH_DATE_TO_USE}')
curr_collected_outputs_folder = Path(output_path_dicts['neuron_replay_stats_table']['.csv']).resolve().parent
filesystem_path_folder_contents_widget(curr_collected_outputs_folder)

failed for file path: W:\Data\KDIBA\gor01\two\2006-6-12_16-53-46\output\pipeline_results.h5, table_key: /kdiba/gor01/two/2006-6-12_16-53-46/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/two/2006-6-12_16-53-46/neuron_identities/table``. Skipping.
failed for file path: W:\Data\KDIBA\pin01\one\11-02_17-46-44\output\pipeline_results.h5, table_key: /kdiba/pin01/one/11-02_17-46-44/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/pin01/one/11-02_17-46-44/neuron_identities/table``. Skipping.
concatenating dataframes from 13 of 15 files
failed for file path: W:\Data\KDIBA\gor01\two\2006-6-12_16-53-46\output\pipeline_results.h5, table_key: /kdiba/gor01/two/2006-6-12_16-53-46/global_computations/long_short_fr_indicies_analysis/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/two/2006-6-12_16-53-46/global_computations/long_short_fr_indicies_analysis/table``. Skipping.
concatenating dat

In [10]:
neuron_replay_stats_table

,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,...,short_RL_pf2D_peak_y,long_LR_pf1D_peak,long_RL_pf1D_peak,short_LR_pf1D_peak,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_uid,neuron_uid,session_datetime
0,kdiba,gor01,one,2006-6-08_14-26-15,nan,nan,SHARED,0.41242440259338126,0.11553338629183138,-0.2968910163015499,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|2,2006-06-08 14:26:15
1,kdiba,gor01,one,2006-6-08_14-26-15,141.38552447193086,198.28087068725637,SHARED,0.2740687245121229,1.3945946124809006,1.1205258879687776,...,142.5757858256191,143.28203601244172,nan,128.10994368835492,nan,-15.172092324086805,nan,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|3,2006-06-08 14:26:15
2,kdiba,gor01,one,2006-6-08_14-26-15,nan,179.3157552821479,RIGHT_ONLY,0.11938205363580597,0.9248959624728122,0.8055139088370062,...,144.79977489932494,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|4,2006-06-08 14:26:15
3,kdiba,gor01,one,2006-6-08_14-26-15,99.66227058069217,202.07389376827808,SHARED,2.3260457715375873,2.0126115307154993,-0.31343424082208804,...,140.81377851894104,234.31458995696255,93.97273595915962,200.17738222776723,109.14482828324643,-34.137207729195325,15.172092324086805,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|5,2006-06-08 14:26:15
4,kdiba,gor01,one,2006-6-08_14-26-15,nan,nan,SHARED,25.389639299524944,12.75463574941881,-12.635003550106134,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|6,2006-06-08 14:26:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,kdiba,pin01,one,fet11-01_12-58-54,180.571251387898,176.84413809844182,SHARED,1.2275172634201952,1.3169564137781686,0.08943915035797345,...,146.95940159297726,55.712956191115126,59.44006948057133,130.25522198023924,100.4383156645896,74.54226578912412,40.998246184018264,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|28,2009-11-01 12:58:54
782,kdiba,pin01,one,fet11-01_12-58-54,57.57651283584323,102.3018723093177,SHARED,0.9299910929136757,0.8388929037677681,-0.0910981891459075,...,147.1121365511247,122.80099540132682,115.34676882241442,152.61790171697646,126.52810869078303,29.816906315649646,11.18133986836861,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|29,2009-11-01 12:58:54
783,kdiba,pin01,one,fet11-01_12-58-54,120.93743875659872,128.39166533551114,SHARED,0.9521868562955006,0.8423839925259716,-0.10980286376952897,...,144.91230460949143,223.4330542166444,238.3415073744692,204.79748776936336,186.16192132208232,-18.635566447281036,-52.17958605238687,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|30,2009-11-01 12:58:54
784,kdiba,pin01,one,fet11-01_12-58-54,236.4779507297411,nan,LEFT_ONLY,1.156580968409006,1.026152466402774,-0.13042850200623213,...,146.59615215828077,63.16718277002754,96.71120237513338,115.34676882241442,96.71120237513338,52.17958605238688,0.0,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|31,2009-11-01 12:58:54


In [ ]:

# [a_sess.context for a_sess in good_session_concrete_folders]
# session_uid
neuron_replay_stats_table.dtypes
neuron_replay_stats_table = neuron_replay_stats_table.fillna(np.nan)
neuron_replay_stats_table

.infer_objects()
neuron_replay_stats_table = neuron_replay_stats_table.convert_dtypes()
neuron_replay_stats_table
numeric_columns = ['long_LR_pf1D_peak', 'short_LR_pf1D_peak', 'long_RL_pf2D_peak_x', 'long_RL_pf2D_peak_y']
for a_column in numeric_columns:
    neuron_replay_stats_table[a_column] = neuron_replay_stats_table[a_column].str.replace('<NA>', "np.nan", regex=False) #.replace('<NA>', np.nan) #.fillna(np.nan)
# .astype(float)

neuron_replay_stats_table

In [6]:
#TODO 2024-06-12 11:18: - [ ] HACK: reads back in the exported `neuron_replay_stats_table` CSV to fix an issue with pd.NA and np.nan not reading a NaNs and the dtypes of the columns being messed up on load:

neuron_replay_stats_table: pd.DataFrame = pd.read_csv(Path('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs/2024-06-12_Apogee/neuron_replay_stats_table.csv'), na_values=['', 'nan', 'np.nan','<NA>'],
                                        dtype={
                                            'long_pf_peak_x': 'float64', 
                                            'short_pf_peak_x': 'float64',
                                            'long_LR_pf2D_peak_x': 'float64',
                                            'long_LR_pf2D_peak_y': 'float64',
                                            'long_RL_pf2D_peak_x': 'float64',
                                            'long_RL_pf2D_peak_y': 'float64',
                                            'short_LR_pf2D_peak_x': 'float64',
                                            'short_LR_pf2D_peak_y': 'float64',
                                            'short_RL_pf2D_peak_x': 'float64',
                                            'short_RL_pf2D_peak_y': 'float64',
                                            'long_LR_pf1D_peak': 'float64',
                                            'long_RL_pf1D_peak': 'float64',
                                            'short_LR_pf1D_peak': 'float64',
                                            'short_RL_pf1D_peak': 'float64',
                                            'peak_diff_RL_pf1D_peak': 'float64',
                                            'peak_diff_LR_pf1D_peak': 'float64'
                                        }, index_col='neuron_uid')
neuron_replay_stats_table


,Unnamed: 0,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,...,short_RL_pf2D_peak_x,short_RL_pf2D_peak_y,long_LR_pf1D_peak,long_RL_pf1D_peak,short_LR_pf1D_peak,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_uid,session_datetime
neuron_uid,,,,,,,,,,,,,,,,,,,,,
kdiba|gor01|one|2006-6-08_14-26-15|2,0,kdiba,gor01,one,2006-6-08_14-26-15,NaN,NaN,SHARED,0.412424,0.115533,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15
kdiba|gor01|one|2006-6-08_14-26-15|3,1,kdiba,gor01,one,2006-6-08_14-26-15,141.385524,198.280871,SHARED,0.274069,1.394595,...,195.336656,142.575786,143.282036,NaN,128.109944,NaN,-15.172092,NaN,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15
kdiba|gor01|one|2006-6-08_14-26-15|4,2,kdiba,gor01,one,2006-6-08_14-26-15,NaN,179.315755,RIGHT_ONLY,0.119382,0.924896,...,177.874096,144.799775,NaN,NaN,NaN,NaN,NaN,NaN,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15
kdiba|gor01|one|2006-6-08_14-26-15|5,3,kdiba,gor01,one,2006-6-08_14-26-15,99.662271,202.073894,SHARED,2.326046,2.012612,...,111.319822,140.813779,234.314590,93.972736,200.177382,109.144828,-34.137208,15.172092,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15
kdiba|gor01|one|2006-6-08_14-26-15|6,4,kdiba,gor01,one,2006-6-08_14-26-15,NaN,NaN,SHARED,25.389639,12.754636,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kdiba|pin01|one|fet11-01_12-58-54|28,781,kdiba,pin01,one,fet11-01_12-58-54,180.571251,176.844138,SHARED,1.227517,1.316956,...,142.356312,146.959402,55.712956,59.440069,130.255222,100.438316,74.542266,40.998246,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54
kdiba|pin01|one|fet11-01_12-58-54|29,782,kdiba,pin01,one,fet11-01_12-58-54,57.576513,102.301872,SHARED,0.929991,0.838893,...,82.035449,147.112137,122.800995,115.346769,152.617902,126.528109,29.816906,11.181340,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54
kdiba|pin01|one|fet11-01_12-58-54|30,783,kdiba,pin01,one,fet11-01_12-58-54,120.937439,128.391665,SHARED,0.952187,0.842384,...,122.538156,144.912305,223.433054,238.341507,204.797488,186.161921,-18.635566,-52.179586,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import TruncationCheckingResults

truncation_checking_aclus_dict, jonathan_firing_rate_analysis_result.neuron_replay_stats_df = truncation_checking_result.build_truncation_checking_aclus_dict(neuron_replay_stats_df=jonathan_firing_rate_analysis_result.neuron_replay_stats_df)

# 'is_rate_extrema', 'is_refined_exclusive', 'is_refined_LxC', 'is_refined_SxC'
'is_long_peak_left_cap', 'is_long_peak_right_cap', 'is_long_peak_either_cap'


def _add_cell_remapping_category(neuron_replay_stats_df):
        """ yanked from `_perform_long_short_endcap_analysis to compute within the batch processing notebook
        """
        loaded_track_limits = deepcopy(owning_pipeline_reference.sess.config.loaded_track_limits) # {'long_xlim': array([59.0774, 228.69]), 'short_xlim': array([94.0156, 193.757]), 'long_ylim': array([138.164, 146.12]), 'short_ylim': array([138.021, 146.263])}
        x_midpoint: float = owning_pipeline_reference.sess.config.x_midpoint
        pix2cm: float = owning_pipeline_reference.sess.config.pix2cm

        ## INPUTS: loaded_track_limits
        print(f'loaded_track_limits: {loaded_track_limits}')


        long_xlim = loaded_track_limits['long_xlim']
        # long_ylim = loaded_track_limits['long_ylim']
        short_xlim = loaded_track_limits['short_xlim']
        # short_ylim = loaded_track_limits['short_ylim']

        occupancy_midpoint: float = x_midpoint # 142.7512402496278 # 150.0
        left_cap_x_bound: float = (long_xlim[0] - x_midpoint) #-shift by midpoint - 72.0 # on long track
        right_cap_x_bound: float = (long_xlim[1] - x_midpoint) # 72.0 # on long track
        min_significant_remapping_x_distance: float = 40.0 # from long->short track

        ## STATIC:
        # occupancy_midpoint: float = 142.7512402496278 # 150.0
        # left_cap_x_bound: float = -72.0 # on long track
        # right_cap_x_bound: float = 72.0 # on long track
        # min_significant_remapping_x_distance: float = 40.0 # from long->short track

        # Modifies the `jonathan_firing_rate_analysis_result.neuron_replay_stats_df` in-place instead of creating a copy:
        # neuron_replay_stats_df = deepcopy(jonathan_firing_rate_analysis_result.neuron_replay_stats_df)

        # Extract the peaks of the long placefields to find ones that have peaks outside the boundaries
        long_pf_peaks = neuron_replay_stats_df[neuron_replay_stats_df['has_long_pf']]['long_pf_peak_x'] - occupancy_midpoint # this shift of `occupancy_midpoint` is to center the midpoint of the track at 0. 
        is_left_cap = (long_pf_peaks < left_cap_x_bound)
        is_right_cap = (long_pf_peaks > right_cap_x_bound)
        # is_either_cap =  np.logical_or(is_left_cap, is_right_cap)

        # Adds ['is_long_peak_left_cap', 'is_long_peak_right_cap', 'is_long_peak_either_cap'] columns: 
        neuron_replay_stats_df['is_long_peak_left_cap'] = False
        neuron_replay_stats_df['is_long_peak_right_cap'] = False
        neuron_replay_stats_df.loc[is_left_cap.index, 'is_long_peak_left_cap'] = is_left_cap # True
        neuron_replay_stats_df.loc[is_right_cap.index, 'is_long_peak_right_cap'] = is_right_cap # True

        neuron_replay_stats_df['is_long_peak_either_cap'] = np.logical_or(neuron_replay_stats_df['is_long_peak_left_cap'], neuron_replay_stats_df['is_long_peak_right_cap'])

        # adds ['LS_pf_peak_x_diff'] column
        neuron_replay_stats_df['LS_pf_peak_x_diff'] = neuron_replay_stats_df['long_pf_peak_x'] - neuron_replay_stats_df['short_pf_peak_x']

        cap_cells_df = neuron_replay_stats_df[np.logical_and(neuron_replay_stats_df['has_long_pf'], neuron_replay_stats_df['is_long_peak_either_cap'])]
        num_total_endcap_cells = len(cap_cells_df)

        # "Disppearing" cells fall below the 1Hz firing criteria on the short track:
        disappearing_endcap_cells_df = cap_cells_df[np.logical_not(cap_cells_df['has_short_pf'])]
        num_disappearing_endcap_cells = len(disappearing_endcap_cells_df)
        print(f'num_disappearing_endcap_cells/num_total_endcap_cells: {num_disappearing_endcap_cells}/{num_total_endcap_cells}')

        non_disappearing_endcap_cells_df = cap_cells_df[cap_cells_df['has_short_pf']] # "non_disappearing" cells are those with a placefield on the short track as well
        num_non_disappearing_endcap_cells = len(non_disappearing_endcap_cells_df)
        print(f'num_non_disappearing_endcap_cells/num_total_endcap_cells: {num_non_disappearing_endcap_cells}/{num_total_endcap_cells}')

        # Classify the non_disappearing cells into two groups:
        # 1. Those that exhibit significant remapping onto somewhere else on the track
        non_disappearing_endcap_cells_df['has_significant_distance_remapping'] = (np.abs(non_disappearing_endcap_cells_df['LS_pf_peak_x_diff']) >= min_significant_remapping_x_distance) # The most a placefield could translate intwards would be (35 + (pf_width/2.0)) I think.
        num_significant_position_remappping_endcap_cells = len(non_disappearing_endcap_cells_df[non_disappearing_endcap_cells_df['has_significant_distance_remapping'] == True])
        print(f'num_significant_position_remappping_endcap_cells/num_non_disappearing_endcap_cells: {num_significant_position_remappping_endcap_cells}/{num_non_disappearing_endcap_cells}')

        # 2. Those that seem to remain where they were on the long track, perhaps being "sampling-clipped" or translated adjacent to the platform. These two subcases can be distinguished by a change in the placefield's length (truncated cells would be a fraction of the length, although might need to account for scaling with new track length)
        minorly_changed_endcap_cells_df = non_disappearing_endcap_cells_df[non_disappearing_endcap_cells_df['has_significant_distance_remapping'] == False]

        significant_distant_remapping_endcap_aclus = non_disappearing_endcap_cells_df[non_disappearing_endcap_cells_df['has_significant_distance_remapping']].index # Int64Index([3, 5, 7, 11, 14, 38, 41, 53, 57, 61, 62, 75, 78, 79, 82, 83, 85, 95, 98, 100, 102], dtype='int64')
        
        return neuron_replay_stats_df

neuron_replay_stats_df = _add_cell_remapping_category(neuron_replay_stats_df=neuron_replay_stats_df)

In [95]:
# required to enable non-blocking interaction:
%gui qt5

import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')
# %matplotlib inline
# %matplotlib auto

In [96]:
## Build a colormap that colors each dot by session:
from matplotlib.colors import ListedColormap

# Assuming neuron_replay_stats_table is your DataFrame
unique_sessions = np.unique(neuron_replay_stats_table['session_name'])
num_sessions = len(unique_sessions)

# Create a colormap for each unique session
cmap = plt.cm.get_cmap('tab10', num_sessions)

color_by_session_color_dict = {session: cmap(i) for i, session in enumerate(unique_sessions)}

# Assign colors based on session_name and create a new column
neuron_replay_stats_table['color'] = neuron_replay_stats_table['session_name'].map(color_by_session_color_dict)

## Builds a new colormap from the index of the dataframe (neuron_uid) to the color
complute_unique_unit_id_colors_map = {row.neuron_uid:row.color for row in neuron_replay_stats_table.reset_index().itertuples(index=False)}

C:\Users\pho\AppData\Local\Temp\ipykernel_36272\3508143324.py:9: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('tab10', num_sessions)


## Required for Interactive Plotting and figure displace

In [97]:
from pyphocorehelpers.geometry_helpers import BoundsRect
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import plot_bidirectional_track_remapping_diagram, AclusYOffsetMode, _plot_track_remapping_diagram

## all the grid_bin_bounds are effectively the same, so this doesn't matter much:
# session_uid_to_grid_bin_bounds_dict = {a_sess.context.get_description(separator="|", include_property_names=False):UserAnnotationsManager.get_hardcoded_specific_session_override_dict().get(a_sess.context, {}).get('grid_bin_bounds', None) for a_sess in good_session_concrete_folders}
session_uid_to_grid_bin_bounds_dict: Dict[IdentifyingContext, BoundsRect] = {a_sess.context.get_description(separator="|", include_property_names=False):BoundsRect.init_from_grid_bin_bounds(UserAnnotationsManager.get_hardcoded_specific_session_override_dict().get(a_sess.context, {}).get('grid_bin_bounds', None)) for a_sess in good_session_concrete_folders}
# session_uid_to_grid_bin_bounds_dict
# [str(v) for v in np.unique(neuron_replay_stats_table.session_uid)]

## Extract the LR/RL separated peak location data
active_peaks_df: pd.DataFrame = deepcopy(neuron_replay_stats_table).reset_index()
# Filter rows based on column: 'neuron_type'
active_peaks_df = active_peaks_df[active_peaks_df['neuron_type'] == "pyr"]

# Filter rows based on column: 'track_membership'
active_peaks_df = active_peaks_df[active_peaks_df['track_membership'] == "SHARED"]

## Cell can be either LR/RL or both:

# Drop rows with missing data in columns: 'long_LR_pf1D_peak', 'long_RL_pf1D_peak'
active_peaks_LR_df = deepcopy(active_peaks_df).dropna(subset=['long_LR_pf1D_peak', 'short_LR_pf1D_peak']).reset_index()
# active_peaks_LR_df

active_peaks_RL_df = deepcopy(active_peaks_df).dropna(subset=['long_RL_pf1D_peak', 'short_RL_pf1D_peak']).reset_index()
# active_peaks_RL_df

# LR_column_kwargs = dict(long_column_name='long_LR_pf1D_peak', short_column_name='short_LR_pf1D_peak')
LR_column_kwargs = dict(long_column_name='long_LR_pf2D_peak_x', short_column_name='short_LR_pf2D_peak_x')

# LR_column_kwargs = dict(long_column_name='long_RL_pf1D_peak', short_column_name='short_RL_pf1D_peak')
RL_column_kwargs = dict(long_column_name='long_RL_pf2D_peak_x', short_column_name='short_RL_pf2D_peak_x')

## OUTPUTS: active_peaks_df, active_column_kwargs
# active_peaks_df = deepcopy(active_peaks_LR_df)
# active_column_kwargs = LR_column_kwargs

active_peaks_df = deepcopy(active_peaks_RL_df)
active_column_kwargs = RL_column_kwargs

active_peaks_df = active_peaks_df.set_index('neuron_uid', drop=True)
active_peaks_df

,index,Unnamed: 0,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,...,short_RL_pf2D_peak_y,long_LR_pf1D_peak,long_RL_pf1D_peak,short_LR_pf1D_peak,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_uid,session_datetime,color
neuron_uid,,,,,,,,,,,,,,,,,,,,,
kdiba|gor01|one|2006-6-08_14-26-15|5,3,3,kdiba,gor01,one,2006-6-08_14-26-15,99.662271,202.073894,SHARED,2.326046,...,140.813779,234.314590,93.972736,200.177382,109.144828,-34.137208,15.172092,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,"(0.5490196078431373, 0.33725490196078434, 0.29..."
kdiba|gor01|one|2006-6-08_14-26-15|7,5,5,kdiba,gor01,one,2006-6-08_14-26-15,54.145994,84.490178,SHARED,2.645898,...,144.662540,52.249482,48.456459,90.179713,82.593667,37.930231,34.137208,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,"(0.5490196078431373, 0.33725490196078434, 0.29..."
kdiba|gor01|one|2006-6-08_14-26-15|19,17,17,kdiba,gor01,one,2006-6-08_14-26-15,251.383194,209.659940,SHARED,0.501756,...,NaN,124.316921,143.282036,120.523898,139.489013,-3.793023,-3.793023,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,"(0.5490196078431373, 0.33725490196078434, 0.29..."
kdiba|gor01|one|2006-6-08_14-26-15|33,31,31,kdiba,gor01,one,2006-6-08_14-26-15,141.385524,137.592501,SHARED,0.670474,...,NaN,147.075059,203.970405,135.695990,196.384359,-11.379069,-7.586046,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,"(0.5490196078431373, 0.33725490196078434, 0.29..."
kdiba|gor01|one|2006-6-08_14-26-15|34,32,32,kdiba,gor01,one,2006-6-08_14-26-15,57.939017,92.076224,SHARED,1.501077,...,138.808998,59.835528,52.249482,86.386690,86.386690,26.551162,34.137208,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,"(0.5490196078431373, 0.33725490196078434, 0.29..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kdiba|pin01|one|fet11-01_12-58-54|25,778,778,kdiba,pin01,one,fet11-01_12-58-54,251.386404,206.661044,SHARED,3.110499,...,147.066427,NaN,242.068621,126.528109,107.892542,NaN,-134.176078,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,"(0.09019607843137255, 0.7450980392156863, 0.81..."
kdiba|pin01|one|fet11-01_12-58-54|26,779,779,kdiba,pin01,one,fet11-01_12-58-54,38.940946,113.483212,SHARED,2.290278,...,146.738974,174.980581,182.434808,174.980581,137.709449,0.000000,-44.725359,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,"(0.09019607843137255, 0.7450980392156863, 0.81..."
kdiba|pin01|one|fet11-01_12-58-54|28,781,781,kdiba,pin01,one,fet11-01_12-58-54,180.571251,176.844138,SHARED,1.227517,...,146.959402,55.712956,59.440069,130.255222,100.438316,74.542266,40.998246,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,"(0.09019607843137255, 0.7450980392156863, 0.81..."


In [98]:
# INPUTS: active_column_kwargs, active_peaks_df


# grid_bin_bounds = ((37.0773897438341, 250.69004399129707),
#   (137.925447118083, 145.16448776601297)),
grid_bin_bounds = BoundsRect(xmin=37.0773897438341, xmax=250.69004399129707, ymin=137.925447118083, ymax=145.16448776601297)

kwargs = dict(draw_point_aclu_labels=False, enable_interactivity=False, enable_adjust_overlapping_text=False, unit_id_colors_map=None, is_dark_mode=False, aclus_y_offset_mode=AclusYOffsetMode.RandomJitter, grid_bin_bounds=grid_bin_bounds)

# ## Colored by session
# kwargs = dict(draw_point_aclu_labels=False, enable_interactivity=False, enable_adjust_overlapping_text=False, unit_id_colors_map=complute_unique_unit_id_colors_map, is_dark_mode=False, aclus_y_offset_mode=AclusYOffsetMode.RandomJitter, grid_bin_bounds=grid_bin_bounds)

fix, ax, _outputs_tuple = _plot_track_remapping_diagram(active_peaks_df, **active_column_kwargs, **kwargs)

# leg = ax.legend()
# # leg = ax.legend(['first', 'second'])
# dummies = [ax.plot([], [], ls='-', c=c)[0] for c in color_by_session_color_dict.values()]        
# ax.legend(dummies, color_by_session_color_dict.keys())



disappearing_long_to_short_aclus: ['kdiba|gor01|one|2006-6-08_14-26-15|19'
 'kdiba|gor01|one|2006-6-08_14-26-15|33'
 'kdiba|gor01|one|2006-6-08_14-26-15|60'
 'kdiba|gor01|one|2006-6-08_14-26-15|73'
 'kdiba|gor01|one|2006-6-08_14-26-15|82'
 'kdiba|gor01|one|2006-6-08_14-26-15|85'
 'kdiba|gor01|one|2006-6-08_14-26-15|93'
 'kdiba|gor01|one|2006-6-08_14-26-15|107'
 'kdiba|gor01|one|2006-6-09_1-22-43|13'
 'kdiba|gor01|one|2006-6-09_1-22-43|27'
 'kdiba|gor01|one|2006-6-09_1-22-43|28'
 'kdiba|gor01|one|2006-6-09_1-22-43|32'
 'kdiba|gor01|one|2006-6-09_1-22-43|57'
 'kdiba|gor01|one|2006-6-09_1-22-43|66'
 'kdiba|gor01|one|2006-6-09_1-22-43|91'
 'kdiba|gor01|one|2006-6-09_1-22-43|95'
 'kdiba|gor01|two|2006-6-07_16-40-19|43'
 'kdiba|gor01|two|2006-6-07_16-40-19|49'
 'kdiba|gor01|two|2006-6-08_21-16-25|15'
 'kdiba|gor01|two|2006-6-08_21-16-25|41'
 'kdiba|gor01|two|2006-6-08_21-16-25|76'
 'kdiba|gor01|two|2006-6-08_21-16-25|82'
 'kdiba|vvp01|one|2006-4-09_17-29-30|7'
 'kdiba|vvp01|one|2006-4-09_17-

In [ ]:
good_session_concrete_folders

In [ ]:
# Options
session_identifier_key: str = 'session_name'
# session_identifier_key: str = 'session_datetime'

## !IMPORTANT! Count of the fields of interest using .value_counts(...) and converting to an explicit pd.DataFrame:
# _out_value_counts_df: pd.DataFrame = neuron_replay_stats_table.value_counts(subset=['format_name', 'animal', 'session_name', 'session_datetime','track_membership'], normalize=False, sort=False, ascending=True, dropna=True).reset_index()
# _out_value_counts_df.columns = ['format_name', 'animal', 'session_name', 'session_datetime', 'track_membership', 'count']
_out_value_counts_df: pd.DataFrame = neuron_replay_stats_table.value_counts(subset=['format_name', 'animal', 'session_name', 'session_datetime','track_membership','is_refined_LxC', 'is_refined_SxC'], normalize=False, sort=False, ascending=True, dropna=True).reset_index()
_out_value_counts_df.columns = ['format_name', 'animal', 'session_name', 'session_datetime', 'track_membership', 'is_refined_LxC', 'is_refined_SxC', 'count']
_out_value_counts_df
## Find the time of the first session for each animal:
first_session_time  = _out_value_counts_df.groupby(['animal']).agg(session_datetime_first=('session_datetime', 'first')).reset_index()
## Subtract this initial time from all of the 'session_datetime' entries for each animal:
# Merge the first session time back into the original DataFrame
merged_df = pd.merge(_out_value_counts_df, first_session_time, on='animal')
# Subtract this initial time from all of the 'session_datetime' entries for each animal
merged_df['time_since_first_session'] = merged_df['session_datetime'] - merged_df['session_datetime_first']
merged_df

In [ ]:
import matplotlib.pyplot as plt

point_size = 8
df = _out_value_counts_df.copy()
animals = df['animal'].unique()
track_memberships = df['track_membership'].unique()

fig, axes = plt.subplots(1, len(animals), figsize=(15, 5))

for i, animal in enumerate(animals):
	ax = axes[i]
	subset_df = df[df['animal'] == animal]
	
	for track_membership in track_memberships:
		track_subset_df = subset_df[subset_df['track_membership'] == track_membership]
		ax.plot(track_subset_df['session_datetime'], track_subset_df['count'], label=f'Track: {track_membership}')
		ax.scatter(track_subset_df['session_datetime'], track_subset_df['count'], s=point_size)
		
	ax.set_title(f'Animal: {animal}')
	ax.set_xlabel('Session Datetime')
	ax.set_ylabel('Count')
	ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
_out_value_counts_df

In [ ]:


## See if the number of cells decreases over re-exposures to the track
df = _out_value_counts_df[_out_value_counts_df['animal'] == 'gor01']
# df = _out_value_counts_df[_out_value_counts_df['animal'] == 'pin01']
# df = _out_value_counts_df[_out_value_counts_df['animal'] == 'vvp01']

# Sort by column: 'session_datetime' (ascending)
df = df.sort_values(['session_datetime'])

'LEFT_ONLY'

# df.to_clipboard(index=False)
df

In [ ]:
## Get the number of cells in each session of the animal:
num_LxCs = df[df['track_membership'] == 'LEFT_ONLY']['count'].to_numpy()
num_Shared = df[df['track_membership'] == 'SHARED']['count'].to_numpy()
num_SxCs = df[df['track_membership'] == 'RIGHT_ONLY']['count'].to_numpy()

num_TotalCs = num_LxCs + num_Shared + num_SxCs
num_TotalCs

In [ ]:
# The only safe point to align each session to is the switchpoint (the delta):


In [ ]:
# Each session can be expressed in terms of time from the start of the first session.


In [ ]:
df.plot()


In [ ]:

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
num_sessions: int = 13
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=True)

## Extract output files from all completed sessions:

# 2023-10-06 - `joined_neruon_fri_df` loading

In [ ]:
# BATCH_DATE_TO_USE = '2023-10-05_NewParameters'
BATCH_DATE_TO_USE = '2023-10-07'
all_sessions_joined_neruon_fri_df, out_path = build_and_merge_all_sessions_joined_neruon_fri_df(global_data_root_parent_path, BATCH_DATE_TO_USE)


In [ ]:

joined_neruon_fri_df_basename = f'{BATCH_DATE_TO_USE}_{output_file_prefix}_joined_neruon_fri_df'
AcrossSessionTables.write_table_to_files(joined_neruon_fri_df, global_data_root_parent_path=global_data_root_parent_path, output_basename=joined_neruon_fri_df_basename, include_csv=False)
print(f'>>\t done with {output_file_prefix}')

# 2023-10-04 - Load Saved across-sessions-data, process, and produce figures

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations

# 'collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl'
across_session_result_long_short_recomputed_inst_firing_rate_file = collected_outputs_path.joinpath('across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
across_session_result_long_short_recomputed_inst_firing_rate_file.exists()

'across_session_result_long_short_inst_firing_rate_2024-06-11_GL.pkl'

In [ ]:
## Load the saved across-session results:
# inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
# inst_fr_output_filename = 'across_session_result_long_short_inst_firing_rate.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-21.pkl'
# inst_fr_output_filename=f'across_session_result_handler_{BATCH_DATE_TO_USE}.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-08-09_Test.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl'
# inst_fr_output_filename='across_session_result_long_short_recomputed_inst_firing_rate_2023-10-04-GL-Recomp.pkl'
# inst_fr_output_filename='across_session_result_long_short_recomputed_inst_firing_rate_2023-10-07.pkl'
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# inst_fr_output_load_filepath: Path = Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
inst_fr_output_load_filepath: Path = collected_outputs_path.joinpath('across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
assert inst_fr_output_load_filepath.exists()
inst_fr_output_filename: str = inst_fr_output_load_filepath.name
# across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=inst_fr_output_load_filepath.parent, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables
 
## Load all across-session tables from the pickles:
# output_path_suffix: str = f'2023-10-07'
# output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
output_path_suffix: str = '2024-06-12_GL'
# neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=collected_outputs_path, output_path_suffix=output_path_suffix)
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')

In [ ]:

# graphics_output_dict = AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=global_data_root_parent_path, BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, plotting_enabled=True)
graphics_output_dict = AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=collected_outputs_path, BATCH_DATE_TO_USE=output_path_suffix, plotting_enabled=True)


 #TODO 2023-10-05 11:40: - [ ] Extract the "contrarian cells", the ones that have a strong exclusivity on the laps but the opposite tendency on the replays


## 2023-11-15 - For manual testing

In [ ]:
# ## Load the saved across-session results:
# Outputs: across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list, neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table

inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables

## Load all across-session tables from the pickles:
output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')
# neuron_replay_stats_table


In [ ]:
across_sessions_instantaneous_frs_list

# 2024-06-11 - Across Session Outputs

In [ ]:
from neuropy.core.user_annotations import UserAnnotationsManager, SessionCellExclusivityRecord
from neuropy.utils.result_context import IdentifyingContext

annotation_man = UserAnnotationsManager()

LxC_uids = []
SxC_uids = []

for a_ctxt in included_session_contexts:
	session_uid = a_ctxt.get_description(separator="|", include_property_names=False)
	session_uid
	session_cell_exclusivity: SessionCellExclusivityRecord = annotation_man.annotations[a_ctxt].get('session_cell_exclusivity', None)
	LxC_uids.extend([f"{session_uid}|{aclu}" for aclu in session_cell_exclusivity.LxC])
	SxC_uids.extend([f"{session_uid}|{aclu}" for aclu in session_cell_exclusivity.SxC])
	
# [a_ctxt.get_description(separator="|", include_property_names=False) for a_ctxt in included_session_contexts]

long_short_fr_indicies_analysis_table['XxC_status'] = 'Shared'
long_short_fr_indicies_analysis_table.loc[np.isin(long_short_fr_indicies_analysis_table.neuron_uid, LxC_uids), 'XxC_status'] = 'LxC'
long_short_fr_indicies_analysis_table.loc[np.isin(long_short_fr_indicies_analysis_table.neuron_uid, SxC_uids), 'XxC_status'] = 'SxC'

long_short_fr_indicies_analysis_table

In [ ]:
## 2023-10-11 - Get the long peak location

long_short_fr_indicies_analysis_table['long_pf_peak_x'] = neuron_replay_stats_table['long_pf_peak_x']
long_short_fr_indicies_analysis_table

In [ ]:
matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
long_short_fr_indicies_analysis_table.plot.scatter(x='long_pf_peak_x', y='x_frs_index', title='Pf Peak position vs. LapsFRI', ylabel='Lap FRI')

long_short_fr_indicies_analysis_table.plot.scatter(x='long_pf_peak_x', y='y_frs_index', title='Pf Peak position vs. ReplayFRI', ylabel='Replay FRI')

 #TODO 2023-10-05 11:40: - [ ] Extract the "contrarian cells", the ones that have a strong exclusivity on the laps but the opposite tendency on the replays


In [ ]:
# long_short_fr_indicies_analysis_table_filename = 'output/2023-10-07_long_short_fr_indicies_analysis_table.csv'
long_short_fr_indicies_analysis_table_filename: str = f'output/{BATCH_DATE_TO_USE}_long_short_fr_indicies_analysis_table.csv'
long_short_fr_indicies_analysis_table.to_csv(long_short_fr_indicies_analysis_table_filename)
print(f'saved: {long_short_fr_indicies_analysis_table_filename}')

# 2023-10-10 - Statistics for `across_sessions_bar_graphs`, analysing `across_session_inst_fr_computation` 

In [ ]:
import scipy.stats as stats
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import pho_stats_perform_diagonal_line_binomial_test, pho_stats_bar_graph_t_tests

binom_test_chance_result = pho_stats_perform_diagonal_line_binomial_test(long_short_fr_indicies_analysis_table)
print(f'binom_test_chance_result: {binom_test_chance_result}')

LxC_Laps_T_result, SxC_Laps_T_result, LxC_Replay_T_result, SxC_Replay_T_result = pho_stats_bar_graph_t_tests(across_session_inst_fr_computation)

# n_total: 823, n_above_diagonal: 457, n_exact_on_diagonal: 0, n_below_diagonal: 366
# binom_test_chance_result: BinomTestResult(k=457, n=823, alternative='two-sided', statistic=0.5552855407047388, pvalue=0.0016893424059938723)
# LxC_Laps_T_result: TtestResult(statistic=12.249237714915296, pvalue=3.8446809431691085e-08, df=12)
# SxC_Laps_T_result: TtestResult(statistic=-12.413163641851535, pvalue=5.768107640498047e-07, df=9)
# LxC_Replay_T_result: TtestResult(statistic=-0.7636543941504783, pvalue=0.4598247399068105, df=12)
# SxC_Replay_T_result: TtestResult(statistic=-3.069767144140489, pvalue=0.01335902191105584, df=9)

# n_total: 823, n_above_diagonal: 457, n_exact_on_diagonal: 0, n_below_diagonal: 366
# binom_test_chance_result: BinomTestResult(k=457, n=823, alternative='two-sided', statistic=0.5552855407047388, pvalue=0.0016893424059938723)
# LxC_Laps_T_result: TtestResult(statistic=12.249237714915296, pvalue=3.8446809431691085e-08, df=12)
# SxC_Laps_T_result: TtestResult(statistic=-12.413163641851535, pvalue=5.768107640498047e-07, df=9)
# LxC_Replay_T_result: TtestResult(statistic=-0.7636543941504783, pvalue=0.4598247399068105, df=12)
# SxC_Replay_T_result: TtestResult(statistic=-3.069767144140489, pvalue=0.01335902191105584, df=9)


## 2023-10-04 - Run `AcrossSessionsVisualizations` corresponding to the PhoDibaPaper2023 figures for all sessions


In [ ]:
## Hacks the `PaperFigureTwo` and `InstantaneousSpikeRateGroupsComputation` 
matplotlib_configuration_update(is_interactive=False, backend='Qt5Agg')
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions, enable_tiny_point_labels=False, enable_hover_labels=False, write_vector_format=True)


In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=False, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=True)


In [ ]:
matplotlib_configuration_update(is_interactive=False, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_long_and_short_firing_rate_replays_v_laps_figure(neuron_replay_stats_table=neuron_replay_stats_table, num_sessions=num_sessions, save_figure=True)


In [ ]:
ann_man = UserAnnotationsManager()
included_annotations = {ctxt:ann_man.annotations[ctxt].get('session_cell_exclusivity', None) for ctxt in included_session_contexts}

all_LxCs = []
all_SxCs = []

for ctxt, an_ann in included_annotations.items():
	session_ctxt_key:str = ctxt.get_description(separator='|', subset_includelist=IdentifyingContext._get_session_context_keys())
	all_LxCs.extend([f"{session_ctxt_key}|{aclu}" for aclu in an_ann.LxC])
	all_SxCs.extend([f"{session_ctxt_key}|{aclu}" for aclu in an_ann.SxC])
	
all_LxCs

In [ ]:
all_SxCs

In [ ]:
across_session_inst_fr_computation.LxC_scatter_props
across_session_inst_fr_computation.SxC_scatter_props

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PaperFigureTwo # used in post_compute_all_sessions_processing


## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=num_sessions) # some global context across all of the sessions, not sure what to put here.

instantaneous_time_bin_size_seconds:float = 0.001

# To correctly aggregate results across sessions, it only makes sense to combine entries at the `.cell_agg_inst_fr_list` variable and lower (as the number of cells can be added across sessions, treated as unique for each session).

## Display the aggregate across sessions:
_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=instantaneous_time_bin_size_seconds) # WARNING: we didn't save this info
_out_fig_2.computation_result = across_session_inst_fr_computation # the result loaded from the file
_out_fig_2.active_identifying_session_ctx = across_session_inst_fr_computation.active_identifying_session_ctx
# Set callback, the only self-specific property
# _out_fig_2._pipeline_file_callback_fn = curr_active_pipeline.output_figure # lambda args, kwargs: self.write_to_file(args, kwargs, curr_active_pipeline)
# _out_fig_2.scatter_props_fn = _return_scatter_props_fn

In [ ]:
LxC_aclus = _out_fig_2.computation_result.LxC_aclus
SxC_aclus = _out_fig_2.computation_result.SxC_aclus

LxC_aclus

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import FigureOutputLocation, ContextToPathMode, FileOutputManager # used in post_compute_all_sessions_processing
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PaperFigureTwo # used in post_compute_all_sessions_processing

registered_output_files = {}

def output_figure(final_context: IdentifyingContext, fig, write_vector_format:bool=True, write_png:bool=True, debug_print=True):
    """ outputs the figure using the provided context. """
    from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_and_write_to_file
    def register_output_file(output_path, output_metadata=None):
        """ registers a new output file for the pipeline """
        print(f'register_output_file(output_path: {output_path}, ...)')
        registered_output_files[output_path] = output_metadata or {}

    fig_out_man = FileOutputManager(figure_output_location=FigureOutputLocation.DAILY_PROGRAMMATIC_OUTPUT_FOLDER, context_to_path_mode=ContextToPathMode.HIERARCHY_UNIQUE)
    active_out_figure_paths = build_and_write_to_file(fig, final_context, fig_out_man, write_vector_format=write_vector_format, write_png=write_png, register_output_file_fn=register_output_file)
    return active_out_figure_paths, final_context


# Set callback, the only self-specific property
_out_fig_2._pipeline_file_callback_fn = output_figure

In [ ]:
_out_fig_2.computation_result.Fig2_Laps_FR

In [ ]:
_out_fig_2.computation_result.Fig2_Replay_FR

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=False, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
active_context = global_multi_session_context.adding_context_if_missing(run_date=BATCH_DATE_TO_USE)

_fig_2_theta_out, _fig_2_replay_out = _out_fig_2.display(active_context=active_context, title_modifier_fn=lambda original_title: f"{original_title} ({num_sessions} sessions)",
														defer_show=True, save_figure=True, enable_tiny_point_labels=False)

# _out_fig_2.perform_save()

# _out_fig_2.perform_save(global_multi_session_context, _fig_2_theta_out)




In [ ]:
debug_print: bool = False
enable_neptune: bool = True

if enable_neptune:
    import neptune # for logging progress and results
    from neptune.types import File
    from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner, AutoValueConvertingNeptuneRun, set_environment_variables 

    ## Gets the notebook filepath for Neptune:
    import IPython
    from pyphocorehelpers.programming_helpers import IPythonHelpers
    notebook_filepath: str = IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())
    assert Path(notebook_filepath).resolve().exists(), f"found notebook filepath: '{notebook_filepath}' does not exist"

	
    # notebook_filepath

	# '/home/halechr/repos/Spike3D/BatchInteractiveProcessing_2024-06-11_GL.ipynb'

    neptune_kwargs = {'project':"commander.pho/PhoDibaLongShortAcrossSessions",
    'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}
            
    neptuner = Neptuner(project_name=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])


    if neptuner.run is None:
        # neptuner.run = neptune.init_run(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath]) # see git_ref=GitRef(repository_path="/path/to/repo")
        # Add the session_context properties to the run: {'format_name': 'kdiba', 'animal': 'vvp01', 'exper_name': 'two', 'session_name': '2006-4-09_16-40-54'}

        neptuner.run = AutoValueConvertingNeptuneRun(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath])

        # Create an AutoValueConvertingNeptuneRun and copy the attributes
        # neptuner.run = AutoValueConvertingNeptuneRun(base_run._client, base_run._uuid, base_run._url)
    
        params = {'BATCH_DATE_TO_USE': BATCH_DATE_TO_USE, "run_workstation": "GL"}
        neptuner.run["parameters"] = params

        neptuner.outputs = neptuner.run['outputs']
        neptuner.figures = neptuner.outputs['figures']

    neptuner_run = neptuner.run
    
    # run = neptune.init_run(source_files=["**/*.dvc"])

    # # Pre-execution dataframe view:
    # run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"

else:
    # no neptune:
    neptuner = None    
    neptuner_run = None

In [ ]:
for a_save_path, a_save_dict in registered_output_files.items():
	a_save_dict['fig']
	a_save_dict['context']
	neptuner.figures.upload(File(a_save_path.resolve().as_posix()))

In [ ]:

neptuner_run[f'output_files/across_sessions_fig_2'] = _out_fig_2

In [ ]:
if (neptuner is not None) and (neptuner_run is not None):
    neptuner.stop()